In [1]:
import pandas as pd
import numpy as np
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g4
g4=rdflib.Graph()
g4.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_ministry_4.nt',format='nt')
print(len(g4))

38871


In [3]:
#************Change Path to files************
state_ut_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/state_new.csv",engine='python') 
state_ut_data=state_ut_data.replace(np.nan,"",regex=True)

In [4]:
state_ut_data.head()

,Cat1,Org1,Cat2,Cat3/Org2,Org3
0,State & UTs,['Andaman and Nicobar Island (UT)'],Apex Bodies,"[""Governor's Office(Raj Bhavan) (1)""]","[""Lt. Gen. Governor's Office Andaman and Nicob..."
1,State & UTs,['Andaman and Nicobar Island (UT)'],Apex Bodies,['Official Portal of State Government (1)'],['Andaman and Nicobar Administration (UT)' 'ht...
2,State & UTs,['Andaman and Nicobar Island (UT)'],Apex Bodies,['Online Citizen Services Portal (1)'],['Online Citizen Services Portal Andaman and N...
3,State & UTs,['Andaman and Nicobar Island (UT)'],Department,['Agriculture Department Andaman & Nicobar'],[]
4,State & UTs,['Andaman and Nicobar Island (UT)'],Department,['Andaman & Nicobar Tourism'],[]


In [5]:
state_ut_data.loc[state_ut_data['Cat2']=="Apex Bodies",'Cat2']='State Apex Body'
state_ut_data.loc[state_ut_data['Cat2']=="Department",'Cat2']='State Department'

In [6]:
#Function:Changing plural to singular
def change_cat(col_nam,df):
    singular_plural_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Singular_Plural.csv") 
    for i in range(singular_plural_data.shape[0]):
        val=singular_plural_data.iloc[i,0]
        new_val=singular_plural_data.iloc[i,1]
        df.loc[(df[col_nam]).apply(lambda x: x.strip())==val,col_nam]=new_val
    return(df)

In [7]:
#Function Call:Changing plural to singular
state_ut_data=change_cat('Cat1',state_ut_data)
state_ut_data=change_cat('Cat2',state_ut_data)
state_ut_data=change_cat('Cat3/Org2',state_ut_data)

In [8]:
#Adding column State_UT_NCT and removing info from column 'Org1'
def entity(row):
    if '(UT)' in row['Org1']:
        return('Union Territory')
    elif '(NCT)' in row['Org1']:
        return('National Capital Territory')
    else:
        return('State')
state_ut_data['state_ut_nct']=state_ut_data.apply(lambda row: entity(row ), axis=1)

In [9]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [10]:
pd.set_option('display.max_colwidth', 500)#housekeeping

In [11]:
#finding n_org max
global n_org
org_list=[]
for s,p,o in g4.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    org_list.append(num_i) #list of n_org identifier values
org_list.sort() #sorting the list
n_org=org_list[-1] #the current maximum value of n_org

#No need to find n_cat max, just one skos:category- topGOT
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g4.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [12]:
print(n_org)
print(n_cat)

4248
22


In [13]:
# #Clean org1/org2/org3 and return list
# def org_scrub(org_n):
#     org_n=org_n.replace('[','')
#     org_n=org_n.replace(']','')
#     ls=list(org_n)
#     l_scrub=[]
#     i=0
#     while(i<len(ls)):
#         if ls[i] in ["\'",'\"']:
#             sch_char=ls[i]
#             #l_scrub=[]
#             for j in range(i+1,len(ls)):
#                 if ls[j]==sch_char:
#                     val=ls[i:j+1]
#                     val_s=""
#                     for x in val:
#                         val_s=val_s+x
#                     val_s=val_s.replace(sch_char,"")
#                     val_s=val_s.strip()
#                     if val_s!=',':
#                         l_scrub.append(val_s)
#                     i=j
#         i=i+1
#     return(l_scrub)

In [14]:
def org_scrub(org_n):
    l_scrub=[]
    val=org_n
    val=str(val)
    val=val.replace('[',"")
    val=val.replace(']',"")
    if len(val)==0:
        return(l_scrub)
    #print(val)
    pos_url=val.find('http')
    #print(pos_url)
    if pos_url!=-1:
        val_1=val[:pos_url]+'|'+val[pos_url:]
        pos_end_url=val_1[pos_url+1:].find("'")
        val_2=val_1[:pos_url+1+pos_end_url]+'|'+val_1[pos_url+1+pos_end_url:]
    else:
        sch_ch=val[0]
        pos_end=val[1:].find(sch_ch)
        #print(val[:pos_end+2])
        val_2=val[:pos_end+2]+"|"+val[pos_end+2:]
        #print(val_2)
    org_ls=val_2.split('|')
    
    for i in range(len(org_ls)):
        ele=org_ls[i]
        ele_ls=list(ele)
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[0] in ["'",'"',',',' '] and len(ele_ls)>0:
                ele_ls.pop(0)
                #print(ele_ls)
            if len(ele_ls)>0:
                if ele_ls[0] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[-1] in ["'",'"',',',' ']:
                ele_ls.pop(-1)
            if len(ele_ls)>0:
                if ele_ls[-1] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        ele_j=''.join(ele_ls)
        l_scrub.append(ele_j)
    return(l_scrub)

In [15]:
#create fetch function for cat_id
def fetch_n_cat(cat):
    global n_cat
    global g4
    cat_id=""
    for s,p,o in g4.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        #adding relevant triples
#         g4.add((cat_id,skos.broader, URIRef(bhcatres+'topOT')))
#         g4.add((URIRef(bhcatres+'topOT'),skos.broaderOf,cat_id))
        g4.add((cat_id,RDF.type,skos.Concept))
        g4.add((cat_id,skos.prefLabel, Literal(cat)))
        g4.add((cat_id,skos.altLabel, Literal(cat)))
        g4.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

#create fetch function for org_id
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g4
    global n_org
    org_id=""
    for s,p,o in g4.triples((None, bhorg.hasName,Literal(org_nam))):
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g4.add((org_id, bhorg.hasName, Literal(org_nam)))
        g4.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g4.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g4.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g4.add((org_id,RDF.type,bhorg.Organisation))
        g4.add((org_id,org.identifier,Literal(n_org)))
        g4.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [16]:
#create fetch function for org_id for cat3_org2
def fetch_n_org_cat3_org2(org_nam,org_url="",org_abb="",org1_id=""):
    global g4
    global n_org
    org_id=""
    for s,p,o in g4.triples((None, bhorg.hasName,Literal(org_nam))):
        if (s,org.subOrganizationOf,org1_id) in g4:
            org_id=s
        else:
            org_id=""
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g4.add((org_id, bhorg.hasName, Literal(org_nam)))
        g4.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g4.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g4.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g4.add((org_id,RDF.type,bhorg.Organisation))
        g4.add((org_id,org.identifier,Literal(n_org)))
        g4.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [17]:
#fetch_n_rg for org1_an
def fetch_an_org1(org_nam,org_url="",org_abb=""):
    global g4
    global n_org
    org_id=""
    #fetch cat_id for "District"
    for a,b,c in g4.triples((None, skos.prefLabel,Literal('District'))):
        cat_id_district=a
    for s,p,o in g4.triples((None, bhorg.hasName,Literal(org_nam))):
        if (s, org.classification, cat_id_district) in g4:
            org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g4.add((org_id, bhorg.hasName, Literal(org_nam)))
        g4.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g4.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g4.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g4.add((org_id,RDF.type,bhorg.Organisation))
        g4.add((org_id,org.identifier,Literal(n_org)))
        g4.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [18]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g4
    if not (cat2_id, skos.broader,cat1_id) in g4:
        g4.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g4:
        g4.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g4
    if not (org1_id,org.hasSubOrganization,org2_id) in g4:
        g4.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g4:
        g4.add((org2_id,org.subOrganizationOf,org1_id))

In [19]:
#state_ut_data.shape[0]
for i in range(state_ut_data.shape[0]):
#Cat1,Org1,Cat2,Cat3/Org2,Org3,State_UT_NCT
    cat1=state_ut_data.iloc[i,0]
    org1=state_ut_data.iloc[i,1]
    cat2=state_ut_data.iloc[i,2]
    cat3_org2=state_ut_data.iloc[i,3]
    org3=state_ut_data.iloc[i,4]
    state_ut_nct=state_ut_data.iloc[i,5]
    
    #check if cat1_id already exists, else create
    cat1_id=fetch_n_cat(cat1)
    
    #check if ((cat1 broader cat21)), ((cat21 broaderOf cat1)) exist, else create
    cat_govt=URIRef(bhcatres+'21')
    fetch_cat_broader(cat_govt,cat1_id)
    
    #check if cat2_id already exists, else create
    cat2_id=fetch_n_cat(cat2)
    
    #check if ((cat2 broader cat1)), ((cat1 broaderOf cat2)) exist, else create
    fetch_cat_broader(cat1_id,cat2_id)
    
    #check if state_ut_nct already exists, else create
    state_ut_nct_id=fetch_n_cat(state_ut_nct)
    
    #check if ((state_ut_nct broader cat1)), ((cat1 broaderOf state_ut_nct)) exist, else create
    #check if ((state_ut_nct broader cat2)), ((cat2 broaderOf state_ut_nct)) exist, else create
    fetch_cat_broader(cat1_id,state_ut_nct_id)
    fetch_cat_broader(state_ut_nct_id,cat2_id)
    
    #****Creating cat3 tuples***
    if cat2 in ["Board / Undertaking","Attached & Subordinate Office"]:
        cat3_id=fetch_n_cat(cat3_org2)
        fetch_cat_broader(cat2_id,cat3_id)
        #org2_id=org1_id
    elif cat2=="Local Body":
        ulb_id=fetch_n_cat("Urban Local Body")
        rlb_id=fetch_n_cat("Rural Local Body")
        fetch_cat_broader(cat2_id,rlb_id)
        fetch_cat_broader(cat2_id,ulb_id)
        
        #add altLabels ULB and RLB
        if (ulb_id, skos.altLabel,Literal('ULB')) not in g4:
            g4.add((ulb_id,skos.altLabel,Literal('ULB')))
        if (rlb_id, skos.altLabel,Literal('RLB')) not in g4:
            g4.add((rlb_id,skos.altLabel,Literal('RLB')))
        #org2_id=org1_id
        if cat3_org2 in ["Municipal Corporation","Municipality / Council / Board","Town Panchayat"]:
            cat3_id=fetch_n_cat(cat3_org2)
            fetch_cat_broader(ulb_id,cat3_id)
        elif cat3_org2 == "Zilla/Taluk/ Village Panchayat":
            cat3_id=fetch_n_cat(cat3_org2)
            fetch_cat_broader(rlb_id,cat3_id)   
    
    #creating triples for org1,org2 and org3
    org1=org1.replace(' (UT)',"")
    org1=org1.replace(' (NCT)',"")
    cat3_org2=cat3_org2.replace('["]',"")    
    org3=org3.replace('["]',"")
    org3=org3.replace('[]',"")
    
    if cat2== "District":
        #org1
        org1_ls=org_scrub(org1)
        org1_id=fetch_n_org(org1_ls[0])
        #check and create org1 classification tuples
        if (org1_id,org.classification,cat1_id) not in g4:
            g4.add((org1_id,org.classification,cat1_id))
        if(org1_id,org.classification,state_ut_nct_id) not in g4:
            g4.add((org1_id,org.classification,state_ut_nct_id))
        #org2
        org_nam=""
        org_url=""
        org_abb=""
        if state_ut_nct not in ['Union Territory','National Capital Territory']:
            org2_ls=org_scrub(cat3_org2)
            if (len(org2_ls))>0: 
                if len(org2_ls)==1:
                    org_nam=org2_ls[0]
#                 elif len(org2_ls)==2:
#                     org_nam=org2_ls[0]
#                     org_url=org2_ls[1]
                elif len(org2_ls)==2:
                    org_nam=org2_ls[0]
                    if org2_ls[1].find('http')>-1:
                        org_url=org2_ls[1]
                    else:
                        org_abb=org2_ls[1]
                else:
                    org_nam=org2_ls[0]
                    org_url=org2_ls[1]
                    org_abb=org2_ls[2]
                org2_id=fetch_an_org1(org_nam,org_url,org_abb)
                g4.add((org2_id,org.classification,cat2_id))
                fetch_org_subOrg(org1_id,org2_id)
        else: #state_ut_nct in ['Union Territory','National Capital Territory']
            org2_ls=org_scrub(cat3_org2)
            if (len(org2_ls))>0: 
                if len(org2_ls)==1:
                    org_nam=org2_ls[0]
#                 elif len(org2_ls)==2:
#                     org_nam=org2_ls[0]
#                     org_url=org2_ls[1]
                elif len(org2_ls)==2:
                    org_nam=org2_ls[0]
                    if org2_ls[1].find('http')>-1:
                        org_url=org2_ls[1]
                    else:
                        org_abb=org2_ls[1]
                else:
                    org_nam=org2_ls[0]
                    org_url=org2_ls[1]
                    org_abb=org2_ls[2]
                org2_id=fetch_an_org1(org_nam,org_url,org_abb)
                g4.add((org2_id,org.classification,cat2_id))
                ##check if the name of org1 and name of org2 are same, if not--->add sugOrg tuples
                #fetching name of org1
#                 org1_nam=g4.value(org1_id,bhorg.hasName)
#                 if org1_nam==org_nam: #check if its same
#                     print(org1_nam,' not equal ',org_nam)
#                     print(len(org1_nam),":",len(org_nam))
#                     fetch_org_subOrg(org1_id,org2_id)
    
    if cat2== "Local Body":
        org1_ls=org_scrub(org1)
        org1_id=fetch_n_org(org1_ls[0])
        #check and create org1 tuples
        if (org1_id,org.classification,cat1_id) not in g4:
            g4.add((org1_id,org.classification,cat1_id))
        if(org1_id,org.classification,state_ut_nct_id) not in g4:
            g4.add((org1_id,org.classification,state_ut_nct_id))
        #check and create org3 tuples
        org_nam=""
        org_url=""
        org_abb=""
        org3_ls=org_scrub(org3)
        if (len(org3_ls))>0: 
            if len(org3_ls)==1:
                org_nam=org3_ls[0]
#             elif len(org3_ls)==2:
#                 org_nam=org3_ls[0]
#                 org_url=org3_ls[1]
            elif len(org3_ls)==2:
                org_nam=org3_ls[0]
                if org1_ls[1].find('http')>-1:
                    org_url=org3_ls[1]
                else:
                    org_abb=org3_ls[1]
            else:
                org_nam=org3_ls[0]
                org_url=org3_ls[1]
                org_abb=org3_ls[2]
            org3_id=fetch_n_org(org_nam,org_url,org_abb)
            g4.add((org3_id,org.classification,cat3_id))
            fetch_org_subOrg(org1_id,org3_id)
    
    if cat2 in ["State Apex Body","State Department"]:
        org1_ls=org_scrub(org1)
        org1_id=fetch_n_org(org1_ls[0])
        #check and create org1 tuples
        if (org1_id,org.classification,cat1_id) not in g4:
            g4.add((org1_id,org.classification,cat1_id))
        if(org1_id,org.classification,state_ut_nct_id) not in g4:
            g4.add((org1_id,org.classification,state_ut_nct_id))
        
        if cat3_org2!="":
            org2_ls=org_scrub(cat3_org2)
            org_nam=""
            org_url=""
            org_abb=""
            if (len(org2_ls))>0: 
                if len(org2_ls)==1:
                    org_nam=org2_ls[0]
                elif len(org2_ls)==2:
                    org_nam=org2_ls[0]
                    org_url=org2_ls[1]
                else:
                    org_nam=org2_ls[0]
                    org_url=org2_ls[1]
                    org_abb=org2_ls[2]
                org2_id=fetch_n_org_cat3_org2(org_nam,org_url,org_abb,org1_id)
                if(org2_id,org.classification,cat2_id) not in g4:
                    g4.add((org2_id,org.classification,cat2_id))
                fetch_org_subOrg(org1_id,org2_id)
        if org3!="":
            org3_ls=org_scrub(org3)
            if (len(org3_ls))>0: 
                if len(org3_ls)==1:
                    org_nam=org3_ls[0]
#                 elif len(org3_ls)==2:
#                     org_nam=org3_ls[0]
#                     org_url=org3_ls[1]
                elif len(org3_ls)==2:
                    org_nam=org3_ls[0]
                    if org3_ls[1].find('http')>-1:
                        org_url=org3_ls[1]
                    else:
                        org_abb=org3_ls[1]
                else:
                    org_nam=org3_ls[0]
                    org_url=org3_ls[1]
                    org_abb=org3_ls[2]
                org3_id=fetch_n_org(org_nam,org_url,org_abb)
                if (org3_id,org.classification,cat2_id) not in g4:
                    g4.add((org3_id,org.classification,cat2_id))
                fetch_org_subOrg(org2_id,org3_id)
    if cat2 in ["Board / Undertaking","Attached & Subordinate Office"]:
        #check and create org1 tuples
        org1_ls=org_scrub(org1)
        org1_id=fetch_n_org(org1_ls[0])
        if (org1_id,org.classification,cat1_id) not in g4:
            g4.add((org1_id,org.classification,cat1_id))
        if (org1_id,org.classification,state_ut_nct_id) not in g4:
            g4.add((org1_id,org.classification,state_ut_nct_id))
        if org3!="":
            org3_ls=org_scrub(org3)
            org_nam=""
            org_url=""
            org_abb=""
            
            if (len(org3_ls))>0: 
                if len(org3_ls)==1:
                    org_nam=org3_ls[0]
                elif len(org3_ls)==2:
                    org_nam=org3_ls[0]
                    if org3_ls[1].find('http')>-1:
                        org_url=org3_ls[1]
                    else:
                        org_abb=org3_ls[1]
                else:
                    org_nam=org3_ls[0]
                    org_url=org3_ls[1]
                    org_abb=org3_ls[2]
                org3_id=fetch_n_org(org_nam,org_url,org_abb)
                if(org3_id,org.classification,cat3_id) not in g4:
                    g4.add((org3_id,org.classification,cat3_id))
                fetch_org_subOrg(org1_id,org3_id) 

In [20]:
print(len(g4))

101439


In [21]:
#Removing/Replacing tuples

#Changing altName for UT, NCT, RLB, ULB
for s,p,o in g4.triples((None,skos.altLabel,Literal("Union Territory"))):
    g4.add((s,skos.altLabel,Literal("UT")))
    g4.remove((s,skos.altLabel,Literal("Union Territory")))
for s,p,o in g4.triples((None,skos.altLabel,Literal("National Capital Territory"))):
    g4.add((s,skos.altLabel,Literal("NCT")))
    g4.remove((s,skos.altLabel,Literal("National Capital Territory")))
for s,p,o in g4.triples((None,skos.altLabel,Literal("Urban Local Body"))):
    g4.add((s,skos.altLabel,Literal("ULB")))
    g4.remove((s,skos.altLabel,Literal("Urban Local Body")))
for s,p,o in g4.triples((None,skos.altLabel,Literal("Rural Local Body"))):
    g4.add((s,skos.altLabel,Literal("RLB")))
    g4.remove((s,skos.altLabel,Literal("Rural Local Body")))

In [22]:
for s,p,o in g4.triples((None,skos.altLabel,Literal("ULB"))):
    for s1,p1,o1 in g4.triples((s,None,None)):
        print(s1,p1,o1)

http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#broaderOf http://semanticwebindia.in/bharathi/res/cat/237
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#broaderOf http://semanticwebindia.in/bharathi/res/cat/236
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#broaderOf http://semanticwebindia.in/bharathi/res/cat/240
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#altLabel ULB
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#prefLabel Urban Local Body
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#broader http://semanticwebindia.in/bharathi/res/cat/233
http://semanticwebindia.in/bharathi/res/cat/234 http://www.w3.org/2004/02/skos/core#in

In [23]:
#Testing
for s,p,o in g4.triples((None, skos.prefLabel,Literal("Zilla/Taluk/ Village Panchayat"))):
    print(s,p,o)
    for s1,p1,o1 in g4.triples((s,None,None)):
        print(s1,p1,o1)

http://semanticwebindia.in/bharathi/res/cat/238 http://www.w3.org/2004/02/skos/core#prefLabel Zilla/Taluk/ Village Panchayat
http://semanticwebindia.in/bharathi/res/cat/238 http://www.w3.org/2004/02/skos/core#altLabel Zilla/Taluk/ Village Panchayat
http://semanticwebindia.in/bharathi/res/cat/238 http://www.w3.org/2004/02/skos/core#prefLabel Zilla/Taluk/ Village Panchayat
http://semanticwebindia.in/bharathi/res/cat/238 http://www.w3.org/2004/02/skos/core#inScheme http://semanticwebindia.in/bharathi/res/cat/OrganisationType
http://semanticwebindia.in/bharathi/res/cat/238 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
http://semanticwebindia.in/bharathi/res/cat/238 http://www.w3.org/2004/02/skos/core#broader http://semanticwebindia.in/bharathi/res/cat/235


In [24]:
g4.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_stateUT_5.nt')

In [31]:
for s,p,o in g4.triples((None,bhorg.hasName,Literal("Puducherry"))):
    for s1,p1,o1 in g4.triples((s,org.classification,None)):
        print(s1,p1,o1)
    for s2,p2,o2 in g4.triples((s,org.subOrganizationOf,None)):
        print(s2,p2,o2)

http://semanticwebindia.in/bharathi/res/org/18056 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/225
http://semanticwebindia.in/bharathi/res/org/18056 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/223
http://semanticwebindia.in/bharathi/res/org/18141 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/225
http://semanticwebindia.in/bharathi/res/org/18141 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/230
http://semanticwebindia.in/bharathi/res/org/18141 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/223


In [26]:
u=URIRef("http://semanticwebindia.in/bharathi/res/cat/224")
i=0
for s,p,o in g4.triples((None,None,u)):
    i+=1
    #print(s,p,o)
print(i)

612


In [27]:
for s,p,o in g4.triples((None,bhorg.hasName,Literal("Mahe"))):
    for s1,p1,o1 in g4.triples((s,org.classification,None)):
        print(s1,p1,o1)

http://semanticwebindia.in/bharathi/res/org/18140 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/230


In [28]:
for s,p,o in g4.triples((None,None,Literal("State & Union Territory"))):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/cat/223 http://www.w3.org/2004/02/skos/core#altLabel State & Union Territory
http://semanticwebindia.in/bharathi/res/cat/223 http://www.w3.org/2004/02/skos/core#prefLabel State & Union Territory


In [29]:
u=URIRef('http://semanticwebindia.in/bharathi/res/org/18114')
val=g4.value(u,bhorg.hasName)
print(val==Literal("Puducherry"))
print(type(val))

False
<class 'rdflib.term.Literal'>


In [30]:
state_ut_data.iloc[0,0]

'State & Union Territory'